In [ ]:
!pip install yfinance ta --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from ta.trend import MACD
import numpy as np

plt.style.use('ggplot')

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select all 10 .csv files


Saving BAJFINANCE.csv to BAJFINANCE.csv
Saving BHARTIARTL.csv to BHARTIARTL.csv
Saving HDFCBANK.csv to HDFCBANK.csv
Saving INFY.csv to INFY.csv
Saving ITC.csv to ITC.csv
Saving LT.csv to LT.csv
Saving RELIANCE.csv to RELIANCE.csv
Saving SUNPHARMA.csv to SUNPHARMA.csv
Saving TATACHEM.csv to TATACHEM.csv
Saving TATAMOTORS.csv to TATAMOTORS.csv


In [ ]:
import pandas as pd
import numpy as np
from ta.trend import MACD
from sklearn.metrics import accuracy_score
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

# Your stock CSVs
stock_files = {
    'HDFC Bank': 'HDFCBANK.csv',
    'Reliance': 'RELIANCE.csv',
    'Infosys': 'INFY.csv',
    'ITC': 'ITC.csv',
    'Tata Motors': 'TATAMOTORS.csv',
    'Sun Pharma': 'SUNPHARMA.csv',
    'L&T': 'LT.csv',
    'Bharti Airtel': 'BHARTIARTL.csv',
    'Bajaj Finance': 'BAJFINANCE.csv',
    'Tata Chemicals': 'TATACHEM.csv'
}

macd_results = []

for stock_name, filename in stock_files.items():
    try:
        # Load and preprocess
        df = pd.read_csv(filename)
        df.columns = [col.lower() for col in df.columns]
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.set_index('date', inplace=True)
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df.dropna(subset=['close'], inplace=True)

        # MACD calculation
        macd = MACD(close=df['close'])
        df['macd'] = macd.macd()
        df['macd_signal'] = macd.macd_signal()

        # Signal: Buy when MACD crosses above Signal, Sell when crosses below
        df['signal'] = 0
        df.loc[(df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1)), 'signal'] = 1
        df.loc[(df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1)), 'signal'] = -1

        # Backtest
        df['return'] = df['close'].pct_change()
        df['strategy_return'] = df['signal'].shift(1) * df['return']
        df.dropna(inplace=True)

        # Accuracy
        actual_direction = (df['return'] > 0).astype(int)
        predicted_direction = (df['signal'].shift(1) > 0).astype(int)
        predicted_direction = predicted_direction[df.index.isin(actual_direction.index)]
        accuracy = accuracy_score(actual_direction, predicted_direction)

        # Sharpe Ratio
        sharpe = (df['strategy_return'].mean() / df['strategy_return'].std()) * np.sqrt(252) if df['strategy_return'].std() != 0 else 0

        # Max Drawdown
        cum_returns = (1 + df['strategy_return']).cumprod()
        peak = cum_returns.cummax()
        drawdown = (peak - cum_returns) / peak
        max_dd = drawdown.max()

        # Store results
        macd_results.append({
            'Stock': stock_name,
            'Accuracy': round(accuracy, 2),
            'Sharpe Ratio': round(sharpe, 2),
            'Max Drawdown (%)': round(max_dd * 100, 2)
        })

        print(f"✅ {stock_name} MACD strategy completed.")

    except Exception as e:
        print(f"❌ {stock_name} failed: {e}")


✅ HDFC Bank MACD strategy completed.
✅ Reliance MACD strategy completed.
✅ Infosys MACD strategy completed.
✅ ITC MACD strategy completed.
✅ Tata Motors MACD strategy completed.
✅ Sun Pharma MACD strategy completed.
✅ L&T MACD strategy completed.
✅ Bharti Airtel MACD strategy completed.
✅ Bajaj Finance MACD strategy completed.
✅ Tata Chemicals MACD strategy completed.


In [ ]:
# Create DataFrame
macd_df = pd.DataFrame(macd_results)

# Save to CSV
macd_df.to_csv('macd_results.csv', index=False)

# Download
files.download('macd_results.csv')

# Display on screen
macd_df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Stock,Accuracy,Sharpe Ratio,Max Drawdown (%)
0,HDFC Bank,0.48,0.08,22.65
1,Reliance,0.49,0.18,25.49
2,Infosys,0.48,0.17,24.27
3,ITC,0.50,0.36,14.37
4,Tata Motors,0.51,-0.04,46.23
5,Sun Pharma,0.49,0.06,26.17
6,L&T,0.50,0.16,17.62
7,Bharti Airtel,0.50,0.02,22.62
8,Bajaj Finance,0.48,0.50,28.20
9,Tata Chemicals,0.48,-0.18,24.34
